# Dow Jones Industrial Average.
We get the dataset from Kaggle:
https://www.kaggle.com/datasets/mnassrib/dow-jones-industrial-average

* The purpose of this notebook is to perform feature engineering tasks in
order to augment the data in this table so we can create a more powerful and
efficient linear regression model.

# Step 1: Reading the dataset


In [1]:
import pandas as pd
import numpy as np

# Storing the data in a dataframe and renaming Price and Vol to Close and Volume
df_raw = pd.read_csv("./dataset/doj_historical.csv", index_col="Date", parse_dates=True)

df_raw.rename(columns = {"Price" : "Close", "Vol.": "Volume" }, inplace = True)
# Evaluating the columns and the content of the dataframe
df_raw.head(5)

# We have six original columns and we'll transform them to get more dimensions

,Close,Open,High,Low,Volume,Change %
Date,,,,,,
2019-12-31,"28,538.44","28,414.64","28,547.35","28,376.49",193.34M,0.27%
2019-12-30,"28,462.14","28,654.76","28,664.69","28,428.98",185.07M,-0.64%
2019-12-27,"28,645.26","28,675.34","28,701.66","28,608.98",184.93M,0.08%
2019-12-26,"28,621.39","28,539.46","28,624.10","28,535.15",155.97M,0.37%
2019-12-24,"28,515.45","28,572.57","28,576.80","28,503.21",95.29M,-0.13%


In [2]:
df_raw.drop("Change %", axis = 1, inplace = True)

In [3]:
df_raw.columns

Index(['Close', 'Open', 'High', 'Low', 'Volume'], dtype='object')

In [4]:
# Counting the number of samples (each row is a day in the stock market)
df_raw.count(axis = 0)
# We have 2766 days worth of data

Close     2766
Open      2766
High      2766
Low       2766
Volume    2766
dtype: int64

In [5]:
# The dimensions of our raw dataset
len(df_raw.columns), len(df_raw.values)

(5, 2766)

In [6]:
# Find rows with '-' in any of the columns
filtered_df = df_raw[df_raw.apply(lambda row: row.str.contains('-')).any
(axis=1)]
print("\nRows with '-' symbol:")
print(filtered_df)



Rows with '-' symbol:
                Close       Open       High        Low Volume
Date                                                         
2010-05-13  10,782.95  10,896.61  10,952.84  10,752.72      -
2010-05-12  10,896.91  10,742.15  10,941.88  10,725.81      -
2010-05-11  10,748.26  10,780.00  10,888.30  10,653.71      -
2010-05-10  10,785.14  10,386.18  10,880.14  10,386.18      -
2010-05-07  10,380.43  10,519.42  10,622.27  10,221.50      -
...               ...        ...        ...        ...    ...
2009-01-09   8,599.18   8,738.80   8,800.45   8,541.75      -
2009-01-08   8,742.46   8,769.94   8,807.14   8,593.52      -
2009-01-07   8,769.70   8,996.94   8,996.94   8,690.45      -
2009-01-06   9,015.10   8,954.57   9,175.19   8,868.07      -
2009-01-05   8,952.89   9,027.13   9,093.47   8,841.70      -

[342 rows x 5 columns]


There seems to be problematic input that caused the following error when
transforming the data into floats:
Error: ValueError: could not convert string to float: '-'
Solution:
1) develop a lambda function to find and replace the values:
2) use a technique to fill these values with either the mean, the mode or the
 medium - Simple imputer


In [7]:
# We'll delete the M for all values and multiply for 1000,
# All other values that contain "-", we'll replace them with nan
df_raw['Volume'] = df_raw['Volume'].apply(lambda x: 1000 * float(x.replace("M",
                                                                        ""))
if x != "-" else float("nan"))

df_raw['Volume']

Date
2019-12-31    193340.0
2019-12-30    185070.0
2019-12-27    184930.0
2019-12-26    155970.0
2019-12-24     95290.0
                ...   
2009-01-09         NaN
2009-01-08         NaN
2009-01-07         NaN
2009-01-06         NaN
2009-01-05         NaN
Name: Volume, Length: 2766, dtype: float64

In [8]:
from sklearn.impute import SimpleImputer

# Use the SimpleImputer to convert all missing values from nan to the mean of
# all other rows
volume_transformed = SimpleImputer(missing_values = np.nan, strategy = "mean")
volume_transformed.fit(df_raw['Volume'].values.reshape(-1, 1))
volume_transformed.transform(df_raw['Volume'].values.reshape(-1, 1))

df_raw['Volume']

Date
2019-12-31    193340.0
2019-12-30    185070.0
2019-12-27    184930.0
2019-12-26    155970.0
2019-12-24     95290.0
                ...   
2009-01-09         NaN
2009-01-08         NaN
2009-01-07         NaN
2009-01-06         NaN
2009-01-05         NaN
Name: Volume, Length: 2766, dtype: float64

In [9]:
# Making sure that all values are of type float since there are some
# characters (commas, suffixes) that will stop us from using the functions
def remove_commas(value):
    """
    This function converts strings in our rows into floats. It also removes
    commas and the M suffix in the column for volume.
    @param value: str, The value in the current row
    @return: float, a clean version of the original data
    """
    return float(value.replace(",", ""))

# Applying the custom function to all elements in the dataframe
df_raw.iloc[:,:-1] = df_raw.iloc[:,:-1].applymap(remove_commas)



C:\Users\52556\AppData\Local\Temp\ipykernel_30148\362144960.py:13: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_raw.iloc[:,:-1] = df_raw.iloc[:,:-1].applymap(remove_commas)


# Step 2: Using Pandas to generate features
* We'll implement a series of functions that directly create features from
the original six financial variables.
* A main function will then call these sub-functions to generate a new
dataset with a total of 37 columns.


In [10]:
def add_original_feature(df, df_new):
    """
    Generate features for a stock/index on historical price and performance.
    It uses pandas functions to generate new columns. Ex: shift() and rolling().
    @param df: dataframe with columns: "Open", "Close", "High", "Low", "Volume", "Adjusted Close"
    @param df_new: dataframe with new features based on the original dataset.
    """
    # Getting the 6 original features
    df_new['open'] = df ['Open']
    df_new['open_1'] = df['Open'].shift(1)
    df_new['close_1'] = df['Close'].shift(1)
    df_new['high_1'] = df['High'].shift(1)
    df_new['low_1'] = df['Low'].shift(1)
    df_new['volume_1'] = df['Volume'].shift(1)

def add_avg_price(df, df_new):
    """
    A sub-function that generates six features related to average close prices:
    @param df: dataframe with columns: "Open", "Close", "High", "Low", "Volume", "Adjusted Close"
    @param df_new: dataframe with 6 new features based on the original dataset.
    """
    # We use the rolling() function to create a window and then apply a
    # function to the specified number of rows. Essentially, we're creating 3
    # new columns with the average prices for a week, a month and a year
    df_new['avg_price_5'] = df['Close'].rolling(5).mean().shift(1)
    df_new['avg_price_30'] = df['Close'].rolling(21).mean().shift(1)
    df_new['avg_price_365'] = df['Close'].rolling(252).mean().shift(1)

    # We create new columns that will store the ratio of change between a
    # week/month, week/year, month/year
    df_new['ratio_avg_price_5_30'] = df_new['avg_price_5'] / df_new['avg_price_30']
    df_new['ratio_avg_price_5_365'] = df_new['avg_price_5'] / df_new['avg_price_365']
    df_new['ratio_avg_price_30_365'] = df_new['avg_price_30'] / df_new['avg_price_365']

def add_avg_volume(df, df_new):
    """
    A sub-function that generates six features related to average volumes:
    @param df: dataframe with columns: "Open", "Close", "High", "Low", "Volume", "Adjusted Close"
    @param df_new: dataframe with 6 new features based on the original dataset.
    """
    # We create three new columns by averaging the change rate of 5, 21 and
    # 252 trading days (week, month, year).
    df_new['avg_volume_5'] = df['Volume'].rolling(5).mean().shift(1)
    df_new['avg_volume_30'] =   df['Volume'].rolling(21).mean().shift(1)
    df_new['avg_volume_365'] = df['Volume'].rolling(252).mean().shift(1)

    # And now we create three new features by getting the ratio of volume
    # change between week/month, week/year, month /year
    df_new['ratio_avg_volume_5_30'] = df_new['avg_volume_5'] / df_new['avg_volume_30']
    df_new['ratio_avg_volume_5_365'] = df_new['avg_volume_5'] / df_new['avg_volume_365']
    df_new['ratio_avg_volume_30_365'] = df_new['avg_volume_30'] / df_new['avg_volume_365']

def add_std_price(df, df_new):
    """
    A sub-function that calculates the standard deviation for the
    price-related features:
    @param df: dataframe with columns: "Open", "Close", "High", "Low", "Volume", "Adjusted Close"
    @param df_new: dataframe with 6 new features based on the original dataset.
    """
    # Using pandas rolling() function, we get the a window for 5, 21 and 252
    # trading days, then we apply the std() function and shift 1 row below
    df_new['std_price_5'] = df['Close'].rolling(5).std().shift(1)
    df_new['std_price_30'] = df['Close'].rolling(21).std().shift(1)
    df_new['std_price_365'] = df['Close'].rolling(252).std().shift(1)
    # Similarly, we create features for the ratio between the newly
    # calculated standard deviations for week/year, week/month and so on
    df_new['ratio_std_price_5_30'] = df_new['std_price_5'] / df_new['std_price_30']
    df_new['ratio_std_price_5_365'] = df_new['std_price_5'] / df_new['std_price_365']
    df_new['ratio_std_price_30_365'] = df_new['std_price_30'] / df_new['std_price_365']

def add_std_volume(df, df_new):
    """
    A sub-function that calculates the standard deviation for the
    volume-based standard deviation features:
    @param df: dataframe with columns: "Open", "Close", "High", "Low", "Volume", "Adjusted Close"
    @param df_new: dataframe with 6 new features based on the original dataset.
    """
    # Using pandas rolling() function, we get the a window for 5, 21 and 252
    # trading days, then we apply the std() function and shift 1 row below
    df_new['std_volume_5'] = df['Volume'].rolling(5).std().shift(1)
    df_new['std_volume_30'] = df['Volume'].rolling(21).std().shift(1)
    df_new['std_volume_365'] = df['Volume'].rolling(252).std().shift(1)
    # Similarly, we create features for the ratio between the newly
    # calculated standard deviations for week/year, week/month and so on
    df_new['ratio_std_volume_5_30'] = df_new['std_volume_5'] / df_new['std_volume_30']
    df_new['ratio_std_volume_5_365'] = df_new['std_volume_5'] / df_new['std_volume_365']
    df_new['ratio_std_volume_30_365'] = df_new['std_volume_30'] / df_new['std_volume_365']

def add_return_feature(df, df_new):
    """
    This function calculates the return for the stock. That is the value of
    the current Closing price minus the Closing price of the previous day
    over the price in the previous day. This function gets the
    return value for the previous day, the past week, past month and past
    year. It also calculates the moving average. In total, we get 7 new features
    @param df: dataframe with columns: "Open", "Close", "High", "Low", "Volume", "Adjusted Close"
    @param df_new: dataframe with 7 new features based on the original dataset.
    """
    # Calculating the return for the previous day, the past week, past month
    # and past year
    df_new['return_1'] = ((df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1).shift(1))
    df_new['return_5'] = ((df['Close'] - df['Close'].shift(5)) / df['Close'].shift(5)).shift(1)
    df_new['return_30'] = ((df['Close'] - df['Close'].shift(21)) / df['Close'].shift(21)).shift(1)
    df_new['return_365'] = ((df['Close'] - df['Close'].shift(252)) / df['Close'].shift(252)).shift(1)

    # Now calculating the moving average for a week, a month and a year
    df_new['moving_avg_5'] = df_new['return_1'].rolling(5).mean().shift(1)
    df_new['moving_avg_30'] = df_new['return_1'].rolling(21).mean().shift(1)
    df_new['moving_avg_365'] = df_new['return_1'].rolling(252).mean().shift(1)

# Step 3: Putting all the sub-functions together.
* Note that the window sizes here are 5, 21, and 252, instead of 7, 30, and 365 representing the weekly, monthly, and yearly window. This is because there are 252 (rounded) trading days in a year, 21 trading days in a month, and 5 in a week.

In [11]:
# Now we create a main function that calls all preceding sub-functions and
# creates a new dataframe with 31 new features
def generate_features(df):
    """
    Generate features for a stock/index based on historical price and
    performance
    @param df: dataframe with columns "Open", "Close", "High", "Low", "Volume", "Adjusted Close"
    @return: dataframe, data set with 31 new features
    """
    df_new = pd.DataFrame()

    # Getting the 6 original features:
    add_original_feature(df, df_new)

    # Add the 31 new features
    add_avg_price(df, df_new)
    add_avg_volume(df, df_new)
    add_std_price(df, df_new)
    add_std_volume(df, df_new)
    add_return_feature(df, df_new)

    # Adding a new column, the target (we'll predict a stock price)
    # dropna : 0, or ‘index’ : Drop rows which contain missing values.
    df_new['close'] = df['Close']
    df_new = df_new.dropna(axis = 0)
    return df_new



In [12]:
# Creating a new, enhanced dataset:
data = generate_features(df_raw)
# The dimensions of our new dataset
print(len(data.columns), len(data.values))

# Taking a look at what the data with the new features looks like
print(data.round(decimals=3).head(5))



38 2171
                open    open_1   close_1    high_1     low_1  volume_1  \
Date                                                                     
2018-12-27  22629.06  23213.61  23062.40  23381.88  22981.33  336510.0   
2018-12-26  21857.73  22629.06  23138.82  23138.89  22267.42  407940.0   
2018-12-24  22317.28  21857.73  22878.45  22878.92  21712.53  433080.0   
2018-12-21  22871.74  22317.28  21792.20  22339.87  21792.20  308420.0   
2018-12-20  23224.12  22871.74  22445.37  23254.59  22396.34  900510.0   

            avg_price_5  avg_price_30  avg_price_365  ratio_avg_price_5_30  \
Date                                                                         
2018-12-27    23171.096     23985.192      26337.481                 0.966   
2018-12-26    23112.228     23916.566      26315.630                 0.966   
2018-12-24    23150.674     23838.006      26292.841                 0.971   
2018-12-21    22839.866     23697.768      26266.161                 0.964   
2018-